In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T

spark = (
    SparkSession
    .builder
    .appName("SparkHiveTest")
    .config("hive.metastore.uris", "thrift://localhost:9083")
    .config("spark.sql.warehouse.dir", "file:///home/jovyan/work/spark-warehouse") # 폴더만 만들어지는 문제
    .enableHiveSupport()
    .getOrCreate()
)

# 5. Spark SQL
## 5.1 DataFrame

* DataFram을 생성하는 방법
    * 기존 RDD를 변환
    * SQL 쿼리를 실행
    * 외부 데이터에서 로드
    
### 5.1.1 RDD에서 DataFrame 생성


* RDD 사용

In [2]:
itPostsRows = spark.sparkContext.textFile("../book-samples/ch05/italianPosts.csv")
itPostsSplit = itPostsRows.map(lambda line: line.split("~"))
itPostsSplit.take(1)

[['4',
  '2013-11-11 18:21:10.903',
  '17',
  "&lt;p&gt;The infinitive tense is commonly used for expressing rules especially in signs (of any kind, not just road signs).&lt;/p&gt;&lt;p&gt;For instance&lt;/p&gt;&lt;blockquote&gt;  &lt;p&gt;Non fumare&lt;br&gt;  Non calpestare il prato&lt;br&gt;  Tenere la destra&lt;/p&gt;&lt;/blockquote&gt;&lt;p&gt;The language &quot;trick&quot; behind this use of the infinitive form is the omission of the clause &lt;em&gt;Si prega di&lt;/em&gt; or equivalent, so the above sentences are read as&lt;/p&gt;&lt;blockquote&gt;  &lt;p&gt;&lt;em&gt;&lt;strong&gt;Si prega di&lt;/em&gt;&lt;/strong&gt; non fumare&lt;br&gt;  &lt;strong&gt;&lt;em&gt;Si prega di&lt;/em&gt;&lt;/strong&gt; non calpestare il prato&lt;br&gt;  &lt;strong&gt;&lt;em&gt;Si prega di&lt;/em&gt;&lt;/strong&gt; tenere la destra&lt;/p&gt;&lt;/blockquote&gt;&lt;p&gt;Such form is not used in everyday's spoken language, as it's a convention used for giving orders and stating rules in an impersonal

In [3]:
itPostsRDD = itPostsSplit.map(lambda x: (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12]))
itPostsDFrame = itPostsRDD.toDF()

In [4]:
itPostsDFrame.where(F.col("_7") == "null").show(10, truncate=True)  # null 이라는 문자

+---+--------------------+---+--------------------+---+--------------------+----+---+---+----+----+---+----+
| _1|                  _2| _3|                  _4| _5|                  _6|  _7| _8| _9| _10| _11|_12| _13|
+---+--------------------+---+--------------------+---+--------------------+----+---+---+----+----+---+----+
|  4|2013-11-11 18:21:...| 17|&lt;p&gt;The infi...| 23|2013-11-10 19:37:...|null|   |   |null|null|  2|1165|
|  2|2013-11-10 20:31:...| 17|&lt;p&gt;Il verbo...|  5|2013-11-10 19:58:...|null|   |   |null|null|  2|1167|
|  0|2013-11-10 22:15:...| 70|&lt;p&gt;&lt;em&g...|  3|2013-11-10 22:15:...|null|   |   |null|null|  2|1169|
|  2|2013-11-10 22:17:...| 17|&lt;p&gt;There's ...|  8|2013-11-10 22:17:...|null|   |   |null|null|  2|1170|
|  1|2013-11-11 09:51:...| 63|&lt;p&gt;As other...|  3|2013-11-11 09:51:...|null|   |   |null|null|  2|1171|
|  1|2013-11-12 23:57:...| 63|&lt;p&gt;The expr...|  1|2013-11-11 10:09:...|null|   |   |null|null|  2|1172|
|  0|2013-11-11 10:

* Pandas 사용

In [5]:
import pandas as pd

it_posts_pd = pd.read_csv("../book-samples/ch05/italianPosts.csv", delimiter="~", header=None)
df = spark.createDataFrame(it_posts_pd.astype(str))
df.where(F.col("6") == "nan").show(10, truncate=True)  # nan 이라는 문자가 채워짐에 주의

+---+--------------------+---+--------------------+---+--------------------+---+---+---+---+---+---+----+
|  0|                   1|  2|                   3|  4|                   5|  6|  7|  8|  9| 10| 11|  12|
+---+--------------------+---+--------------------+---+--------------------+---+---+---+---+---+---+----+
|  4|2013-11-11 18:21:...| 17|&lt;p&gt;The infi...| 23|2013-11-10 19:37:...|nan|nan|nan|nan|nan|  2|1165|
|  2|2013-11-10 20:31:...| 17|&lt;p&gt;Il verbo...|  5|2013-11-10 19:58:...|nan|nan|nan|nan|nan|  2|1167|
|  0|2013-11-10 22:15:...| 70|&lt;p&gt;&lt;em&g...|  3|2013-11-10 22:15:...|nan|nan|nan|nan|nan|  2|1169|
|  2|2013-11-10 22:17:...| 17|&lt;p&gt;There's ...|  8|2013-11-10 22:17:...|nan|nan|nan|nan|nan|  2|1170|
|  1|2013-11-11 09:51:...| 63|&lt;p&gt;As other...|  3|2013-11-11 09:51:...|nan|nan|nan|nan|nan|  2|1171|
|  1|2013-11-12 23:57:...| 63|&lt;p&gt;The expr...|  1|2013-11-11 10:09:...|nan|nan|nan|nan|nan|  2|1172|
|  0|2013-11-11 10:58:...| 18|&lt;p&gt;Wow, wh

#### 5.1.1.4 케이스 클래스를 사용해 RDD를 DataFrame으로 변환

In [6]:
from pyspark.sql import Row
from datetime import datetime

def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S.%f")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def stringToPost(row):
  r = row.split("~")
  return Row(toIntSafe(r[0]), toTimeSafe(r[1]), toIntSafe(r[2]), r[3], toIntSafe(r[4]), toTimeSafe(r[5]), toIntSafe(r[6]),
             toIntSafe(r[7]), r[8], toIntSafe(r[9]), toLongSafe(r[10]), toLongSafe(r[11]), int(r[12]))

rowRDD = itPostsRows.map(lambda x: stringToPost(x))  # 불편해서 안쓰겠음
# rowRDD.toDF() # 안됨

#### 5.1.1.5 스키마를 지정해 RDD를 DataFrame으로 변환

In [7]:
from pyspark.sql.types import *

postSchema = StructType([
  StructField("commentCount", IntegerType(), True),
  StructField("lastActivityDate", TimestampType(), True),
  StructField("ownerUserId", LongType(), True),
  StructField("body", StringType(), True),
  StructField("score", IntegerType(), True),
  StructField("creationDate", TimestampType(), True),
  StructField("viewCount", IntegerType(), True),
  StructField("title", StringType(), True),
  StructField("tags", StringType(), True),
  StructField("answerCount", IntegerType(), True),
  StructField("acceptedAnswerId", LongType(), True),
  StructField("postTypeId", LongType(), True),
  StructField("id", LongType(), False)
  ])

In [8]:
rowRDD = itPostsRows.map(lambda x: stringToPost(x))
itPostsDFStruct = spark.createDataFrame(rowRDD, postSchema)
itPostsDFStruct.printSchema()

root
 |-- commentCount: integer (nullable = true)
 |-- lastActivityDate: timestamp (nullable = true)
 |-- ownerUserId: long (nullable = true)
 |-- body: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- creationDate: timestamp (nullable = true)
 |-- viewCount: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- answerCount: integer (nullable = true)
 |-- acceptedAnswerId: long (nullable = true)
 |-- postTypeId: long (nullable = true)
 |-- id: long (nullable = false)



#### 5.1.1.6 스키마 정보 가져오기

In [9]:
itPostsDFStruct.printSchema()

root
 |-- commentCount: integer (nullable = true)
 |-- lastActivityDate: timestamp (nullable = true)
 |-- ownerUserId: long (nullable = true)
 |-- body: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- creationDate: timestamp (nullable = true)
 |-- viewCount: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- answerCount: integer (nullable = true)
 |-- acceptedAnswerId: long (nullable = true)
 |-- postTypeId: long (nullable = true)
 |-- id: long (nullable = false)



In [10]:
itPostsDFStruct.dtypes

[('commentCount', 'int'),
 ('lastActivityDate', 'timestamp'),
 ('ownerUserId', 'bigint'),
 ('body', 'string'),
 ('score', 'int'),
 ('creationDate', 'timestamp'),
 ('viewCount', 'int'),
 ('title', 'string'),
 ('tags', 'string'),
 ('answerCount', 'int'),
 ('acceptedAnswerId', 'bigint'),
 ('postTypeId', 'bigint'),
 ('id', 'bigint')]

### 5.1.2 기본 DataFrame API

In [11]:
posts_pd = pd.read_csv("../book-samples/ch05/italianPosts.csv", delimiter="~")
posts_pd.columns = ["commentCount", "lastActivityDate", "ownerUserId", "body", "score", "creationDate", "viewCount", "title", "tags", "answerCount", "acceptedAnswerId", "postTypeId", "id"]
posts_df = spark.createDataFrame(posts_pd.astype(str))

In [12]:
posts_df.head()

Row(commentCount='5', lastActivityDate='2013-11-10 20:31:00.177', ownerUserId='12', body="&lt;p&gt;Come credo sia conosciuto da tutti quelli che usano viaggiare con l'automobile, molti italiani hanno uno strano rapporto con gli abbaglianti; alcuni li amano così tanto che preferiscono mantenerli sempre accesi, altri invece li usano per segnalare, se non addirittura per comunicare informazioni di vario genere, dalla presenza di autovelox alla protesta per presunte violazioni del codice della strada.&lt;/p&gt;&lt;p&gt;Al di lá delle considerazioni e dei commenti circa queste abitudini, mi piacerebbe sapere se il verbo &quot;sfanagliare&quot; è normalmente usato, e compreso, in tutte le regioni italiane o se, magari, ci sono altri verbi in uso, purchè simpatici come quello.&lt;/p&gt;&lt;p&gt;Laddove qualcuno non avesse compreso l'uso del aforementioned verbo, ecco un esempio:&lt;/p&gt;&lt;blockquote&gt;  &lt;p&gt;&quot;Ehi!&quot; - dice il marito a sua moglie - &quot;Quello li mi sta sfana

In [13]:
posts_df.where(F.col("body").contains("Italiano")).count()

46

In [14]:
# 필터링
(
    posts_df
    .where((F.col("postTypeId") == 1) & (F.col("acceptedAnswerId").isNull())).count()
)

0

In [15]:
# 상위 10
firstTenQs = (
    posts_df
    .where(F.col("postTypeId") == 1).limit(10)
)

firstTenQs.show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|     61.0|Cosa sapreste dir...| &lt;word-choice&gt;|        1.0|             nan|         1|1166|
|           1|2014-07-25 13:15:...|        154|&lt;p&gt;As part ...|   11|2013-11-10 22:03:...|    187.0|Ironic constructi...|&lt;english-compa...|        4.0|          1170.0|         1|1168|
|           9|2014-01-05 11:13:...|

In [16]:
# 컬림 추가 및 이름 변경
firstTenQsRn = firstTenQs.withColumnRenamed("ownerUserId", "owner")

In [17]:
(
    posts_df.where(F.col("postTypeId") == 1)
    .withColumn("ratio", F.col("viewCount") / F.col("score"))
    .where(F.col("ratio") < 35).show()
)

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+-------------------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|              ratio|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+-------------------+
|           1|2014-07-25 13:15:...|        154|&lt;p&gt;As part ...|   11|2013-11-10 22:03:...|    187.0|Ironic constructi...|&lt;english-compa...|        4.0|          1170.0|         1|1168|               17.0|
|           9|2014-01-05 11:13:...|         63|&lt;p&gt;When I w...|    5|2013-11-11 10:28:...|    122.0|Is &quot;scancell...|&lt;usage&gt;&lt;...| 

In [18]:
# 정렬
posts_df.sort(F.desc("lastActivityDate")).show(10)

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|           2|2014-09-13 22:25:...|        707|&lt;p&gt;Ho osser...|    3|2014-09-12 09:44:...|    102.0|Perché a volte si...|&lt;orthography&g...|        1.0|          2344.0|         1|2343|
|           3|2014-09-13 13:40:...|        193|&lt;p&gt;Due to t...|    1|2014-09-11 01:16:...|      nan|                 nan|                 nan|        nan|             nan|         2|2337|
|           9|2014-09-13 13:40:...|

### 5.1.3 SQL함수로 데이터에 연산 수행
#### 5.1.3.1 내장 스칼라 함수와 내장 집계 함수

* abs, hypot, log, cbrt
* length, trim, concat
* year, date_add
* min, max, count, avg, sum 등

In [19]:
# 가장 오랜시간 논의한 질문
active_period = (
    posts_df
    .filter(posts_df.postTypeId == 1)
    .withColumn("activePeriod", F.datediff(posts_df.lastActivityDate, posts_df.creationDate))
    .orderBy(F.desc("activePeriod"))
)

active_period.columns

['commentCount',
 'lastActivityDate',
 'ownerUserId',
 'body',
 'score',
 'creationDate',
 'viewCount',
 'title',
 'tags',
 'answerCount',
 'acceptedAnswerId',
 'postTypeId',
 'id',
 'activePeriod']

In [20]:
# body 컬럼을 출력
active_period.head().body.replace("&lt;","<").replace("&gt;",">")

'<p>The plural of <em>braccio</em> is <em>braccia</em>, and the plural of <em>avambraccio</em> is <em>avambracci</em>.</p><p>Why are the plural of those words so different, if they both are referring to parts of the human body, and <em>avambraccio</em> derives from <em>braccio</em>?</p>'

In [21]:
posts_df.select(F.avg("score").alias("AVG"), F.max("score").alias("MAX"), F.count("score").alias("CNT")).show()

+-----------------+---+----+
|              AVG|MAX| CNT|
+-----------------+---+----+
|4.144444444444445|  9|1260|
+-----------------+---+----+



#### 5.1.3.2 윈도우 함수

분석 함수들
* first(column), last(column), 
* lag(column, offset, [default]), lead(column, offset, [default])
* ntile(n): 프레임에 포함된 로우를 구룹 n개로 분할하고, 현재 로우가 속한 그룹의 인덱스를 반환
* cume_dist: 프레임에 포함된 로우 중에서 현재 처리하고 있는 로우의 값보다 작거나 같은 다른 로우들이 프레임에서 차지하는 비율
* rank, desnse_rank
* precent_rank
* row_number

윈도우 사양
* partitionBy
* orderBy
* rowsBetween(from, to): 현재 처리하는 로우는 0, 직전 -1, 직후 1로 간주

In [22]:
# 사용자별 최고 점수와 현재 점수의 차이 계산

from pyspark.sql import Window

ownerUserIdPartition = Window.partitionBy("ownerUserId")

(
    posts_df
    .where(F.col("postTypeId") == 1)
    .select(
        F.col("ownerUserId"),
        F.col("acceptedAnswerId"),
        F.col("score"),
        (F.max("score").over(ownerUserIdPartition)).alias("maxPerUser"))
    .withColumn("toMax", F.col("maxPerUser") - F.col("score"))
    .show(10)
)

+-----------+----------------+-----+----------+-----+
|ownerUserId|acceptedAnswerId|score|maxPerUser|toMax|
+-----------+----------------+-----+----------+-----+
|        124|            88.0|    4|         4|  0.0|
|        591|             nan|    3|         4|  1.0|
|        591|          2336.0|    4|         4|  0.0|
|        591|          2158.0|    2|         4|  2.0|
|        232|          2185.0|    6|         6|  0.0|
|        448|             nan|    3|         3|  0.0|
|        132|          1235.0|   14|         7| -7.0|
|        132|             nan|    7|         7|  0.0|
|        132|          1348.0|    7|         7|  0.0|
|        154|          1170.0|   11|        11|  0.0|
+-----------+----------------+-----+----------+-----+
only showing top 10 rows



In [23]:
# 질문의 생성 날짜를 기준으로 바로 전 질문과 바

user_ordered_by_date_partition = Window.partitionBy("ownerUserId").orderBy("creationDate")

(
    posts_df.where(F.col("postTypeId") == 1)
    .select(
        F.col("ownerUserId"),
        F.col("id"),
        F.col("creationDate"),
        F.lag("id", 1).over(user_ordered_by_date_partition).alias("prev"),
        F.lead("id", 1).over(user_ordered_by_date_partition).alias("next"),
    ).show(10, False)
)

+-----------+----+-----------------------+----+----+
|ownerUserId|id  |creationDate           |prev|next|
+-----------+----+-----------------------+----+----+
|124        |86  |2013-11-07 17:43:30.977|null|null|
|591        |2147|2014-08-06 13:25:56.72 |null|2317|
|591        |2317|2014-09-05 15:09:51.65 |2147|2331|
|591        |2331|2014-09-10 02:16:31.523|2317|null|
|232        |2184|2014-08-13 12:31:34.313|null|null|
|448        |2020|2014-06-12 16:10:33.177|null|null|
|132        |1225|2013-11-12 14:14:50.85 |null|1238|
|132        |1238|2013-11-12 20:01:40.093|1225|1265|
|132        |1265|2013-11-14 15:44:50.99 |1238|null|
|154        |1168|2013-11-10 22:03:41.027|null|null|
+-----------+----+-----------------------+----+----+
only showing top 10 rows



### 5.1.3.3 사용자 정의 함수

* 파이썬 사용자는 [Pandas UDF](https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html)를 권장함(속도 저하)

In [24]:
countTags = F.udf(lambda tags: tags.count("&lt;"), IntegerType())
posts_df.filter(posts_df.postTypeId == 1).select("tags", countTags(posts_df.tags).alias("tagCnt")).show(10, False)

+-------------------------------------------------------------------+------+
|tags                                                               |tagCnt|
+-------------------------------------------------------------------+------+
|&lt;word-choice&gt;                                                |1     |
|&lt;english-comparison&gt;&lt;translation&gt;&lt;phrase-request&gt;|3     |
|&lt;usage&gt;&lt;verbs&gt;                                         |2     |
|&lt;usage&gt;&lt;tenses&gt;&lt;english-comparison&gt;              |3     |
|&lt;usage&gt;&lt;punctuation&gt;                                   |2     |
|&lt;usage&gt;&lt;tenses&gt;                                        |2     |
|&lt;history&gt;&lt;english-comparison&gt;                          |2     |
|&lt;idioms&gt;&lt;etymology&gt;                                    |2     |
|&lt;idioms&gt;&lt;regional&gt;                                     |2     |
|&lt;grammar&gt;                                                    |1     |

In [25]:
spark.udf.register("countTags_udf", lambda tags: tags.count("&lt;"), IntegerType())

<function __main__.<lambda>(tags)>

In [26]:
posts_df.filter(posts_df.postTypeId == 1).select("tags").show(10, False)

+-------------------------------------------------------------------+
|tags                                                               |
+-------------------------------------------------------------------+
|&lt;word-choice&gt;                                                |
|&lt;english-comparison&gt;&lt;translation&gt;&lt;phrase-request&gt;|
|&lt;usage&gt;&lt;verbs&gt;                                         |
|&lt;usage&gt;&lt;tenses&gt;&lt;english-comparison&gt;              |
|&lt;usage&gt;&lt;punctuation&gt;                                   |
|&lt;usage&gt;&lt;tenses&gt;                                        |
|&lt;history&gt;&lt;english-comparison&gt;                          |
|&lt;idioms&gt;&lt;etymology&gt;                                    |
|&lt;idioms&gt;&lt;regional&gt;                                     |
|&lt;grammar&gt;                                                    |
+-------------------------------------------------------------------+
only showing top 10 

* pandas udf 사례

In [27]:
import pandas as pd
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import LongType

# Declare the function and create the UDF
@pandas_udf(returnType=IntegerType())
def str_cnt(tags: pd.Series) -> pd.Series:
    return pd.Series(x.count("&lt;") for x in tags)

# str_cnt = pandas_udf(str_cnt, returnType=LongType())

# Execute function as a Spark vectorized UDF
posts_df.select(F.col("tags"), str_cnt(F.col("tags"))).show(10, False)

+-------------------------------------------------------------------+-------------+
|tags                                                               |str_cnt(tags)|
+-------------------------------------------------------------------+-------------+
|&lt;word-choice&gt;                                                |1            |
|nan                                                                |0            |
|&lt;english-comparison&gt;&lt;translation&gt;&lt;phrase-request&gt;|3            |
|nan                                                                |0            |
|nan                                                                |0            |
|nan                                                                |0            |
|nan                                                                |0            |
|&lt;usage&gt;&lt;verbs&gt;                                         |2            |
|nan                                                                |0      

### 5.1.4 결측 값 다루기

In [28]:
clean_posts = posts_df.na.drop()
clean_posts.count()

posts_df.na.fill({"viewCount": 0}).show(10)

posts_df.na.replace(1177, 3000, ["id", "acceptedAnswerId"]).show(10)

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|     61.0|Cosa sapreste dir...| &lt;word-choice&gt;|        1.0|             nan|         1|1166|
|           2|2013-11-10 20:31:...|         17|&lt;p&gt;Il verbo...|    5|2013-11-10 19:58:...|      nan|                 nan|                 nan|        nan|             nan|         2|1167|
|           1|2014-07-25 13:15:...|

### 5.1.5 DataFrame을 Rdd로 변환(생략)
* 사용성 낮음

### 5.1.6 그룹핑

In [29]:
# 집계함수 예제

posts_df.groupBy("ownerUserId", "tags", "postTypeId").count().orderBy(F.desc("ownerUserId")).show()

+-----------+--------------------+----------+-----+
|ownerUserId|                tags|postTypeId|count|
+-----------+--------------------+----------+-----+
|         99|&lt;word-choice&g...|         1|    1|
|         99|&lt;grammar&gt;&l...|         1|    1|
|         98|                 nan|         2|    7|
|         95|                 nan|         2|   29|
|         95|     &lt;grammar&gt;|         1|    3|
|         95|   &lt;etymology&gt;|         1|    1|
|         95|&lt;orthography&g...|         1|    1|
|         91|&lt;grammar&gt;&l...|         1|    1|
|         88|                 nan|         2|    2|
|         88|&lt;word-choice&g...|         1|    1|
|         88|&lt;word-choice&g...|         1|    1|
|         87|                 nan|         2|    6|
|        862|                 nan|         2|    1|
|        855|   &lt;resources&gt;|         1|    1|
|         85|&lt;articles&gt;&...|         1|    1|
|         85|                 nan|         2|    1|
|        846

* 일반적인 그룹핑 함수

In [30]:
posts_df.groupBy(F.col("ownerUserId")).agg(F.max("score")).show(10)

+-----------+----------+
|ownerUserId|max(score)|
+-----------+----------+
|        124|         9|
|        591|         7|
|        307|         0|
|        334|         3|
|        581|         8|
|         15|         8|
|        232|         6|
|        234|         5|
|        448|         3|
|        483|         5|
+-----------+----------+
only showing top 10 rows



In [31]:
simple_df = (
    posts_df
    .where((F.col("ownerUserId") >= 13) & (F.col("ownerUserId") <= 15))
)

* 모든 조합을 대상으로 계산

In [32]:
simple_df.cube("ownerUserId", "tags", "postTypeId").count().show()

+-----------+----+----------+-----+
|ownerUserId|tags|postTypeId|count|
+-----------+----+----------+-----+
|         15| nan|         2|    2|
|         14|null|      null|    2|
|         14| nan|         2|    2|
|         13|null|      null|    1|
|         15|null|      null|    2|
|         13| nan|      null|    1|
|       null|null|      null|    5|
|         15| nan|      null|    2|
|       null| nan|         2|    5|
|         14|null|         2|    2|
|       null|null|         2|    5|
|         13| nan|         2|    1|
|         13|null|         2|    1|
|         15|null|         2|    2|
|         14| nan|      null|    2|
|       null| nan|      null|    5|
+-----------+----+----------+-----+



* 지정된 컬럼 순으로 순열을 사용

In [33]:
simple_df.rollup("ownerUserId", "tags", "postTypeId").count().show()

+-----------+----+----------+-----+
|ownerUserId|tags|postTypeId|count|
+-----------+----+----------+-----+
|         15| nan|         2|    2|
|         14|null|      null|    2|
|         14| nan|         2|    2|
|         13|null|      null|    1|
|         15|null|      null|    2|
|         13| nan|      null|    1|
|       null|null|      null|    5|
|         15| nan|      null|    2|
|         13| nan|         2|    1|
|         14| nan|      null|    2|
+-----------+----+----------+-----+



### 5.1.7 데이터 조인

* spark.sql.shuffle.partitions 변수로 파티션 사이즈를 결정(기본값은 200)

In [34]:
itVotesRaw = spark.read.csv("../book-samples/ch05/italianVotes.csv", sep="~")
itVotesRaw = itVotesRaw.withColumnRenamed('_c0', "id")
itVotesRaw = itVotesRaw.withColumnRenamed('_c1', "postId")
itVotesRaw = itVotesRaw.withColumnRenamed('_c2', "voteTypeId")
itVotesRaw = itVotesRaw.withColumnRenamed('_c3', "creationDate")

itVotesRaw.head()

Row(id='2657', postId='135', voteTypeId='2', creationDate='2013-11-22 00:00:00.0')

In [35]:
posts_df.join(itVotesRaw, "id").show(10)

+----+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+------+----------+--------------------+
|  id|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|postId|voteTypeId|        creationDate|
+----+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+------+----------+--------------------+
|1613|          16|2014-01-30 04:52:...|         95|&lt;p&gt;Solitame...|    6|2014-01-19 10:57:...|    169.0|prestiti dallo sp...|     &lt;grammar&gt;|        2.0|             nan|         1|   143|         2|2013-11-09 00:00:...|
|1614|           0|2014-01-19 21:44:...|         95|&lt;p&gt;the rule...

In [36]:
posts_df.join(itVotesRaw, "id", "outer").show(10)

+----+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+------+----------+--------------------+
|  id|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|postId|voteTypeId|        creationDate|
+----+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+------+----------+--------------------+
|1159|           0|2013-11-10 03:29:...|          4|&lt;p&gt;I was wa...|    8|2013-11-10 03:29:...|      nan|                 nan|                 nan|        nan|             nan|         2|  null|      null|                null|
|1436|           5|2013-11-30 23:16:...|         57|&lt;p&gt;The usag...

## 5.3 SQL 명령
### 5.3.1 테이블 카탈로그와 하이브 메타스토어

#### 5.3.1.1 테이블을 임시로 등록

In [37]:
posts_df.createOrReplaceTempView("posts_temp")

#### 5.3.1.2 테이블을 영구적으로 등록
* saveAsTable() 함수를 사용

In [38]:
# spark.sql("drop table posts")
# spark.sql("drop table votes")

# !rm -r /user/hive/warehouse/posts
# !rm -r /user/hive/warehouse/votes
# !pwd

In [39]:
posts_df.write.option("path", "/home/jovyan/work/spark-warehouse/posts").format("parquet").mode("overwrite").saveAsTable("posts")
itVotesRaw.write.option("path", "/home/jovyan/work/spark-warehouse/votes").format("parquet").mode("overwrite").saveAsTable("votes")

#### 5.3.1.3 스파크 테이블 카탈로그

In [40]:
spark.catalog.listDatabases()

[Database(name='default', description='Default Hive database', locationUri='file:/user/hive/warehouse')]

In [41]:
spark.catalog.listTables() # tableType='MANAGED' 는 스파크가 해당 테이블까지 관리함을 의미

[Table(name='posts', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='votes', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='posts_temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [42]:
spark.catalog.currentDatabase()

'default'

In [43]:
spark.catalog.listColumns("votes")

[Column(name='id', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='postId', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='voteTypeId', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='creationDate', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

In [44]:
spark.catalog.listFunctions()[:5] # SQL 함수 목록

[Function(name='!', description=None, className='org.apache.spark.sql.catalyst.expressions.Not', isTemporary=True),
 Function(name='%', description=None, className='org.apache.spark.sql.catalyst.expressions.Remainder', isTemporary=True),
 Function(name='&', description=None, className='org.apache.spark.sql.catalyst.expressions.BitwiseAnd', isTemporary=True),
 Function(name='*', description=None, className='org.apache.spark.sql.catalyst.expressions.Multiply', isTemporary=True),
 Function(name='+', description=None, className='org.apache.spark.sql.catalyst.expressions.Add', isTemporary=True)]

In [45]:
spark.sql("select * from posts")

DataFrame[commentCount: string, lastActivityDate: string, ownerUserId: string, body: string, score: string, creationDate: string, viewCount: string, title: string, tags: string, answerCount: string, acceptedAnswerId: string, postTypeId: string, id: string]

#### 5.3.1.4 원격 하이브 메타스토어 설정
* hive-site.xml 매개 변수들로 설정
* spark.sql.warehouse.dir 매개변수는 오버라이드 됨
* hive.metastore.warehouse.dir 변수로 기본 저장위치 설정
* 위의 설정이 잘 동작하지 않아 write.option("path", "/some/path/")를 추가함

### 5.3.3. 쓰리프트 서버로 스파크 SQL 접속

In [46]:
# !bash /usr/local/spark/sbin/start-thriftserver.sh

* beeline으로 테스트 시 tables 정보값이 null인 문제(미해결)

## 5.4 DataFrame을 저장하고 불러오기

### 5.4.1 기본 데이터 소스
* JSON
* ORC
* Parquet

### 5.4.2 데이터 저장
* 주요 함수
    * saveAsTable
    * insertInto: 하이브 메타스토어에 존재하는 테이블을 지정해야 함
    * save
* format (default: parquet)
    * orc
    * json
    * jdbc
* mode
    * overwrite 
    * append
    * ignore 
    
### 5.4.3 데이터 불러오기
* read 함수
* jdbc

In [54]:
# !pip install mysql-connector

In [60]:
# import mysql.connector
# conn = mysql.connector.connect(host='mariadb',
#                                database='metastore_db',
#                                user='admin',
#                                port=3306,
#                                password='admin')
# cursor = conn.cursor(prepared=True)
# result = cursor.execute("SELECT * FROM TBLS")
# result

In [48]:
dataframe_mysql = spark.read.format("jdbc").options(
    url="jdbc:mysql://mariadb/metastore_db",
    driver = "com.mysql.cj.jdbc.Driver",
    dbtable = "TBLS",
    user="admin",
    password="admin").load()

dataframe_mysql.show()

+------+-----------+-----+----------------+-----+----------+---------+-----+--------+--------------+------------------+------------------+------------------+
|TBL_ID|CREATE_TIME|DB_ID|LAST_ACCESS_TIME|OWNER|OWNER_TYPE|RETENTION|SD_ID|TBL_NAME|      TBL_TYPE|VIEW_EXPANDED_TEXT|VIEW_ORIGINAL_TEXT|IS_REWRITE_ENABLED|
+------+-----------+-----+----------------+-----+----------+---------+-----+--------+--------------+------------------+------------------+------------------+
|    68| 1605032892|    1|               0| root|      USER|        0|   68|   posts|EXTERNAL_TABLE|              null|              null|             false|
|    69| 1605032892|    1|               0| root|      USER|        0|   69|   votes|EXTERNAL_TABLE|              null|              null|             false|
+------+-----------+-----+----------------+-----+----------+---------+-----+--------+--------------+------------------+------------------+------------------+



* 퀴리 푸시다운
    * DataFrame을 만들기 전에 데이터베이스 자체에서 데이터를 처리하도록 명령하는 것


In [49]:
pushdownQuery = """(SELECT DISTINCT(TBL_NAME) FROM TBLS) AS TBLS """
dbDataFrame = spark.read.format("jdbc")\
    .option("url", "jdbc:mysql://mariadb/metastore_db")\
    .option("dbtable", pushdownQuery)\
    .option("driver", "com.mysql.cj.jdbc.Driver")\
    .option("user", "admin")\
    .option("password", "admin")\
    .load()

dbDataFrame.show()

+--------+
|TBL_NAME|
+--------+
|   posts|
|   votes|
+--------+

